In [3]:
import json
from pprint import pprint
import os
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import torch
import sys
sys.path.append(r'C:\Study\SpeechAcquisitionModel')
from src.reinforcement.goal_directed_model_based_rl.model import SimpleStochasticActorCritic,\
    SimpleStochasticModelDynamics, SimpleDeterministicPolicy, SimpleDeterministicModelDynamics, EnsembleDeterministicModelDynamicsDeltaPredict
from src.reinforcement.goal_directed_model_based_rl.env import VTLEnvWithReferenceTransitionMaskedEntropyScore
from src.reinforcement.goal_directed_model_based_rl.algs.model_based_multi_step_backprop_with_ensemble_classifier import ModelBasedMultiStepBackPropWithEnsembleClassifier
from src.speech_classification.pytorch_conv_lstm import LstmNet, LstmNetEnsemble


C:\Study\SpeechAcquisitionModel\src\reinforcement\goal_directed_model_based_rl


In [ ]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
model_dir = r'C:\Study\SpeechAcquisitionModel\reports\videos\video_ensemble_multi_step_05_17_2019_04_36_PM'

In [ ]:
with open(model_dir+ '/config.json') as data_file:
    kwargs = json.load(data_file)

In [ ]:
device = 'cpu'
kwargs['mbbackprop']['device'] = device

speaker_fname = os.path.join(kwargs['env']['vtl_dir'], 'JD2.speaker')
lib_path = os.path.join(kwargs['env']['vtl_dir'], 'VocalTractLab2.dll')

env = VTLEnvWithReferenceTransitionMaskedEntropyScore(lib_path, speaker_fname, **kwargs['env'])
env.reset(train_mode=True)

In [ ]:
model_dynamics = torch.load(model_dir + '/model_dynamics.pickle')
agent = torch.load(model_dir + '/agent.pickle')


In [2]:
ep_states = []
ep_states_pred = []
ep_actions = []
misses = []

T = len(env.get_current_reference())
state0 = env.reset()
state = state0
env.render()
ep_states.append(state)
ep_states_pred.append(state[:-env.goal_dim])
state = env.normalize(state, env.state_bound)

# axes[0].cla()

score = 0.

agent.eval()
model_dynamics.eval()
step = 0
terminated = False

probs = []
entropies = []

while True:
    state_tensor = torch.tensor(torch.from_numpy(state).float().to(device).view(1, -1), requires_grad=True)
    action = agent(state_tensor).detach().cpu().numpy().squeeze()
    ep_actions.append(action)
    action_denorm = env.denormalize(action, env.action_bound)
    next_state, reward, done, _ = env.step(action_denorm)

    probs.append(reward[0].detach().cpu().numpy())
    entropies.append(reward[1].detach().cpu().numpy())

    action_tensor = torch.tensor(torch.from_numpy(action).float().to(device).view(1,-1), requires_grad=True)
    
    next_state_pred, next_state_pred_std, _ = model_dynamics(state_tensor, action_tensor)
    
    
    saliency_state = []    
    saliency_action = []

    for d in range(next_state_pred.shape[-1]):
#         state_tensor.grad.data.zero_()
#         next_state_pred[:, d].backward(retain_graph=True)
        grad = torch.autograd.grad(next_state_pred[:, d], state_tensor, retain_graph=True)
#         grad = state_tensor.grad.data.detach().numpy()
        saliency_state.append(grad[0].numpy())
    
        grad1 = torch.autograd.grad(next_state_pred[:, d], action_tensor, retain_graph=True)
        saliency_action.append(grad1[0].numpy())
    
    
#         state_tensor.grad.data.zero_()
    saliency_state = np.array(saliency_state).squeeze()
    plt.imshow(saliency_state[-26:, :])
    plt.colorbar()
    plt.title(r'$d S_{pred}^{acoustic} /  dS $')
    plt.show()
    
    saliency_state = np.array(saliency_state).squeeze()
    plt.imshow(saliency_state[:-26, :])
    plt.colorbar()
    plt.title(r'$d S_{pred}^{artic} /  dS $')
    plt.show()
    
    saliency_action = np.array(saliency_action).squeeze()
    plt.imshow(saliency_action)
    plt.colorbar()
    plt.title(r'$d S_{pred} /  dA $')
    plt.show()
    
  
    
    
    
    next_state_pred = env.denormalize(next_state_pred.detach().cpu().numpy().squeeze(), env.state_bound[:-env.goal_dim])

    ep_states.append(next_state)
    ep_states_pred.append(next_state_pred)
    next_state = env.normalize(next_state, env.state_bound)
    env.render()
    
    miss = torch.abs(torch.from_numpy(next_state).float().to(device)[:-env.goal_dim][torch.from_numpy(np.array(env.state_goal_mask, dtype=np.uint8)).byte()] -
                     torch.from_numpy(state).float().to(device)[-env.goal_dim:])

    misses.append(miss[:].max().detach().cpu().numpy())

    if len(misses) > 3 and np.all(np.array(misses[-3:]) > 0.1) and not terminated:
        terminated = True
        res_step = step
        miss_max_idx = np.argmax(miss[:].detach().cpu().numpy())
    elif not terminated:
        score = step
   
    if np.any(done):

        break
    state = next_state
    step += 1

NameError: name 'env' is not defined